# Hand Detection / Gesture Classification Demo Case

This notebook implements a demo case for controlling the JetBot with gestures.
It uses self-trained models for gesture detection and classification as well as the collision avoidance model of the JetBot examples by John Welsh and Chitoku Yato.

## Features
The robot has two modes: *free* and *follow*.

The default is *follow* mode. In this mode the robot will follow the *palm* gesture. <br>
It moves towards or backwards until the detected hand has the right size. 
It will rotate to the left or right until the detected hand is in the middle of the camera image. <br>
The robot will not move into a dangerous situation and will shake itself if it should move towards the *palm* and classifies a dangerous situation at the same time. <br>
If the robot detects the *crawl* gesture, it will switch to *free* mode.

In *free* mode the robot will move around by itself. It tries to avoid collisions and turns if it classifies a dangerous situation.

In both *free* and *follow* mode the robot can classify the *five* gesture.
This will force the robot to spin around.

<img src="images/demo_case_large.png" width="75%">

## Supported gestures
This images taken from the [LaRED](http://mclab.citi.sinica.edu.tw/dataset/lared/lared.html) dataset demonstrate the gestures that are supported by this demo case.
<div style="float:left"><img src="images/crawl.jpg" width="80%"><div style="text-align: center">Crawl</div></div>
<div style="float:left"><img src="images/fist.jpg" width="80%"><div style="text-align: center">Fist</div></div>
<div style="float:left"><img src="images/five.jpg" width="80%"><div style="text-align: center">Five</div></div>
<div style="float:left"><img src="images/palm.jpg" width="80%"><div style="text-align: center">Palm</div></div>
<div style="float:left"><img src="images/peace.jpg" width="80%"><div style="text-align: center">Peace</div></div>
<div style="clear: both"/><br/>

**Configured gesture commands** <br>
The following gestures have been configured to fire an action event.
- *palm*: Follow the hand with the *palm* gesture. Switch to *follow* mode.
- *crawl*: Switch to *free* mode. Drive around freely.
- *five*: Spin around.

## Preconditions
The models have to be placed in the correct directory.
Make sure you have the following folder structure **inside your notebooks directory** where the other JetBot examples can be found. <br>
This will make sure the right dependencies are available.
<pre>
notebooks/
├── basic_motion/
├── collision_avoidance/
└── gestures/ 
    ├── images/
    │   ├── crawl.jpg
    │   ├── ...
    │   └── peace.jpg
    ├── model_alexnet/ 
    │   └── model_collision_avoidance.pth
    ├── model_mobilenet/
    │   ├── saved_model.pb
    │   └── variables/
    │       ├── variables.data-00000-of-00002
    │       ├── variables.data-00001-of-00002
    │       └── variables.index
    ├── model_ssd_mobilenetV2
    │   └── saved_model.pb 
    ├── emobot.py
    └── gesture_demo_case.ipynb
</pre>
You can rename the folders but make sure to edit the paths accordingly when loading the models. The model files may not be renamed.

Make sure that you have NVIDIA `tensorflow-gpu 2.0.0+nv20.1.tf2` for Jetson Nano installed. See the [installation instructions](https://docs.nvidia.com/deeplearning/frameworks/install-tf-jetson-platform/index.html) from NVIDIA. 

## Import the dependencies

In [1]:
import numpy as np
import cv2
import tensorflow as tf
import pathlib
import time

from jetbot import Camera, bgr8_to_jpeg

import torch
import torchvision
import torch.nn.functional as F

from emobot import Emobot

import ipywidgets.widgets as widgets
from IPython.display import display

## Set necessary parameters

In [2]:
INDEX_TO_LABEL = {
    0 :"crawl",
    1 :"fist",
    2 :"five",
    3 :"palm",
    4 :"peace",
}

IMAGE_HEIGHT = 300
IMAGE_WIDTH = 300

THRESHOLD_HAND_DETECTION = 0.8
THRESHOLD_GESTURE_CLASSIFICATION = 0.9

In [3]:
# If you have problems creating the camera, uncomment the next line and run the cell again. This will restart the camera daemon
# !echo jetbot | sudo -S systemctl restart nvargus-daemon

camera = Camera.instance(width=IMAGE_WIDTH, height=IMAGE_HEIGHT)
snapshot = widgets.Image(format='bgr8', width=IMAGE_WIDTH, height=IMAGE_HEIGHT)

robot = Emobot()

## Helper methods
These methods use the gesture detection and classification models.

Some the methods are needed for preprocessing the model input while others use the model output and transform it for usage.

In [4]:
# Define font for the detections and classifications on the camera output
font = cv2.FONT_HERSHEY_SIMPLEX
color = (0,205,205)
fontScale = 0.35
thickness = 1

# Define collision avoidance normalization function
# This has been defined according to the collision_avoidance example by John Welsh & Chitoku Yato
mean = 255.0 * np.array([0.485, 0.456, 0.406])
stdev = 255.0 * np.array([0.229, 0.224, 0.225])
normalize = torchvision.transforms.Normalize(mean, stdev)

def preprocess_collision_avoidance(camera_value):
    """ Preprocesses the camera image for inference.
    
    Also prepares the data on the CUDA device.
    This has been adapted from the collision_avoidance example by John Welsh & Chitoku Yato
    Input:
        camera_value: The current camera frame
    Return:
        x: The Tensor created from the camera image
    """
    
    global device, normalize
    x = camera_value
    x = x.transpose((2, 0, 1))
    x = torch.from_numpy(x).float()
    x = normalize(x)
    x = x.to(device)
    x = x[None, ...]
    return x

def is_collsion(img):
    """ Checks the camera image for a potentially dangerous situation.
    
    Sends the image to the model for inference.
    Input:
        img: The current camera frame
    Returns: 
        boolean: Robot is blocked or free
    """
    
    x = preprocess_collision_avoidance(img)
    y = model_collision(x)

    # We apply the `softmax` function to normalize the output vector so it sums to 1 (which makes it a probability distribution)
    y = F.softmax(y, dim=1)
    prob_blocked = float(y.flatten()[0])
    
    if prob_blocked < 0.5:
        return False
    else:
        return True
    
def filter_detected_boxes(hand_detections, threshold):
    """ Filter the detected hands according to defined threshold.
    
    Only bounding boxes that have been detected above the threshold will be returned.
    Returns: 
        boxes: Bounding boxes of detected hands
        hand_scores: Probability that the detected object is a hand
    """
    boxes_raw = hand_detections['detection_boxes'].numpy()[0]
    scores = hand_detections['detection_scores'].numpy()[0]
    
    boxes = []
    hand_scores = []
    for i, score in enumerate(scores):
        if score > threshold:
            box = boxes_raw[i]
            
            ymin = int(float(box[0])*IMAGE_HEIGHT)
            xmin = int(float(box[1])*IMAGE_WIDTH)
            ymax = int(float(box[2])*IMAGE_HEIGHT)
            xmax = int(float(box[3])*IMAGE_WIDTH)

            score = round(score,2)
            hand_scores.append(score)
            
            boxes.append((xmin, ymin, xmax, ymax))
            
    return boxes, hand_scores

def crop_rect(img, xmin, xmax, ymin, ymax):
    """ Crop the the rectangular image to a square
    
    Input:
        img: Image that contains the bounding box
        xmin, xmax, ymin, ymax: Coordinates of a hand bounding box
    Returns: 
        img: Input image cropped to a square
    """
    x, y = xmin, ymin
    w = xmax - xmin # width
    h = ymax - ymin # height
    
    # Crop a square form
    if w > h:
        y = y - int((w-h)/2)
        h = w    
        # Make sure y is within picture
        y = max(y,0)
        y = min(y, IMAGE_HEIGHT-h)

    elif h > w:
        x = x - int((h-w)/2)
        w = h
        # Make sure x is within picture
        x = max(x,0)
        x = min(x,IMAGE_WIDTH-w)
        
    return img[y:y+h, x:x+w]

def add_boxes_to_img(img, boxes, hand_scores, gesture_names, gesture_scores):
    """ Draw bounding boxes on an image
    
    The calculated probabilities for a hand detection or a gesture classification will be added to the image.
    The gesture names will be drawn on the image.    
    
    Input:
        img: Image that contains the bounding boxes
        boxes: The bounding boxes to draw
        hand_scores: The probability of a detected hand
        gesture_names: The names of classified gestures
        gesture_scores: The probability of a classified gesture
    Returns: 
        img: The input image with drawn bounding boxes and scores
    """
    
    for box, hand_score, gesture_name, gesture_score in zip(boxes, hand_scores, gesture_names, gesture_scores):
        (xmin, ymin, xmax, ymax) = box

        img = cv2.rectangle(img, (xmin, ymin), (xmax, ymax), color, 1)
        img = cv2.putText(img, 'hand ' + str(hand_score), (xmin+2, ymin+10), font, fontScale, color, thickness, cv2.LINE_AA)
        
        if gesture_name:
            img = cv2.putText(img, gesture_name +" "+ gesture_score, (xmin+2, ymax-5), font, fontScale, color, thickness, cv2.LINE_AA)
        
    return img

def get_prediction_gesture(img):
    """ Predict the gesture on the input image
    
    Input:
        img: Image that contains the gesture
    Returns: 
        gesture_name: The name of the predicted gesture
        confidence: The probability for that gesture
    """

    img =  cv2.resize(img, (64,64))
    img = img/255
    img = img.astype('float32')
    input_tensor = tf.convert_to_tensor(img)
    input_tensor = input_tensor[tf.newaxis,...]
    
    predictions = model_classify(input_tensor)
    
    predicted_index = np.argmax(predictions, axis=1)[0]
    confidence = round(np.max(predictions[0]), 2)
    
    if confidence > THRESHOLD_GESTURE_CLASSIFICATION:
        gesture_name = INDEX_TO_LABEL[predicted_index]
    else:
        gesture_name = ""
    
    return gesture_name, str(confidence)

def get_biggest_gesture(boxes, hand_scores, gesture_names, gesture_scores):
    """ Calculate which gesture covers the biggest area
    
    This method will filter for the gesture that covers the biggest area on an image.
    This gesture is presumably the closest to the camera and will therefore take precedence.
    
    Input:
        boxes: Bounding boxes containing the predicted gestures
        hand_scores: Probability for the detected hands for the boxes
        gesture_names: Names of the classified gestures for the boxes
        gesture_scores Probability for the classified gestures for the boxes
    Returns: 
        selected_gesture: Name of the gesture that covers the biggest area
        selected_box: Bounding box of the gesture that covers the biggest area
    """
    selected_gesture = ""
    selected_box = None
    biggest_area = 0
    for box, hand_score, gesture_name, gesture_score in zip(boxes, hand_scores, gesture_names, gesture_scores):
        (xmin, ymin, xmax, ymax) = box
        current_area = (xmax-xmin)*(ymax-ymin)
        if current_area > biggest_area:
            selected_gesture = gesture_name
            selected_box = box
            biggest_area = current_area
            
    return selected_gesture, selected_box

def classify_gestures_for_boxes(boxes):
    """ Classify the gestures for all bounding boxes 
    
    Input:
        boxes: Bounding boxes containing hand detections
    Returns: 
        gesture_names: The names of the classified gestures
        gesture_scores: The probability of the classifed gestures
    """
    gesture_names = []
    gesture_scores = []
    
    for box in boxes:
        (xmin, ymin, xmax, ymax) = box
        cropped_image = crop_rect(img, xmin, xmax, ymin, ymax)
        gesture_name, gesture_score = get_prediction_gesture(cropped_image)
        gesture_names.append(gesture_name)
        gesture_scores.append(gesture_score)
        
    return gesture_names, gesture_scores

def rotate_to_gesture(box):
    """ Rotates the robot towards a gestures
    
    The robot will rotate so that the gesture is in the middle of the camera image.
    
    Input:
        box: Bounding box containing a hand detection
    """
    (xmin, ymin, xmax, ymax) = box

    x_center = (xmax+xmin)/2


    distance_from_center = IMAGE_WIDTH/2 - x_center
    distance_from_center = distance_from_center/(IMAGE_WIDTH/2) # Normalize
    
    if distance_from_center > 0.2:
        robot.rotate_left_by_sight(distance_from_center)
    elif distance_from_center < -0.2:
        robot.rotate_right_by_sight(distance_from_center*-1)


## Load models

Load the hand detection, gesture classificcation and collision avoidance model. <br>
Be patient, as the loading will take some time.

In [5]:
# Hand detection model
model_dir = pathlib.Path("model_ssd_mobilenetV2")
model_detect = tf.saved_model.load(str(model_dir))
model_detect = model_detect.signatures['serving_default']
print("hand detection model loaded")

# Gesture classification model
model_dir = pathlib.Path("model_mobilenet")
model_classify = tf.saved_model.load(str(model_dir))
print("gesture classification model loaded")
# Warmup - Preprocess dummy
get_prediction_gesture(camera.value)

# Collosion avoidance model
model_collision = torchvision.models.alexnet(pretrained=False)
model_collision.classifier[6] = torch.nn.Linear(model_collision.classifier[6].in_features, 2)
model_collision.load_state_dict(torch.load('model_alexnet/model_collision_avoidance.pth'))

device = torch.device('cuda')
model_collision = model_collision.to(device)
print("collosion avoidance model loaded")

INFO:tensorflow:Saver not created because there are no variables in the graph to restore
hand detection model loaded
gesture classification model loaded
collosion avoidance model loaded


## Main Execution Loop

This is the "heart" of the demo case. The general steps are:
1. Get new camera image
2. Check for collision
3. Detect hands
4. Classify gestures
5. Select biggest gesture
6. Act depending on gesture
7. Output detections/classifications to user

To stop the execution click *Interrupt the kernel* in the menu and run `robot.stop()`

<img src="images/demo_case_activity.png" width="100%"><div style="text-align: center">

In [ ]:
prev_time = time.time()
display(snapshot)

MODE = "follow" # free, follow

while True:
    # Read camera feed
    img = camera.value
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Check for dangerous situations
    collision_ahead = is_collsion(img)
    if MODE == "free":
        if collision_ahead:
            robot.left(0.3)
        else:
            robot.forward(0.3)
    
    # Create input Tensor
    input_tensor = tf.convert_to_tensor(img)
    input_tensor = input_tensor[tf.newaxis,...]
    
    # Detect hands
    hand_detections = model_detect(input_tensor)
    boxes, hand_scores = filter_detected_boxes(hand_detections, threshold=THRESHOLD_HAND_DETECTION)

    # Classify gestures
    gesture_names, gesture_scores = classify_gestures_for_boxes(boxes)

    # Select gesture that covers biggest area
    selected_gesture, selected_box = get_biggest_gesture(boxes, hand_scores, gesture_names, gesture_scores)
    
        
    # Act on the classified gesture
    if selected_gesture == "five":
        robot.rotate_left(360);
    elif selected_gesture == "palm":
        MODE = "follow"
        rotate_to_gesture(selected_box)  
        (xmin, ymin, xmax, ymax) = selected_box
        if ymax-ymin < IMAGE_HEIGHT/2.5:
            if collision_ahead:
                robot.shake_it()
            else:
                robot.forward(0.3)
        elif ymax-ymin > IMAGE_HEIGHT/1.7:
            robot.backward(0.25)
        else:
            robot.stop()
    elif selected_gesture == "crawl":
        MODE = "free"
        robot.rotate_right(180)
    elif MODE == "follow":
        robot.stop()
        
    # Draw boxes and labels on displayed image
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    if collision_ahead:
        img = cv2.putText(img, "COLLISION", (20, 20), font, fontScale, color, thickness, cv2.LINE_AA)

    
    add_boxes_to_img(img, boxes, hand_scores, gesture_names, gesture_scores)
    snapshot.value = bytes(cv2.imencode('.jpg', img)[1]) # Same as bgr8_to_jpeg
    
    fps = round(1 / (time.time() - prev_time),1)
    print(str(fps)+"fps", "| Selected gesture:",selected_gesture, "\r", end="")
    prev_time = time.time()
    

Image(value=b'', format='bgr8', height='300', width='300')

In [7]:
# The kernel must be interrupted otherwise this command will not work
robot.stop()